In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as TF
import torchvision.transforms.v2 as TF2
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
def mean(L):
    if len(L) == 0: return None
    return sum(L) / len(L)

In [3]:
tfm = TF.Compose([TF.ToTensor(), TF.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
ds_trn = torchvision.datasets.ImageFolder('./Cifar10/train', transform=tfm)
ds_test = torchvision.datasets.ImageFolder('./Cifar10/test', transform=tfm)

In [4]:
assert len(ds_trn) == 50_000
assert len(ds_test) == 10_000

In [5]:
assert ds_trn.classes == ds_test.classes
num_classes = len(ds_trn.classes)

In [6]:
dl_trn = torch.utils.data.DataLoader(ds_trn, batch_size=32, shuffle=True, drop_last=True)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=64, shuffle=False, drop_last=False)

In [7]:
device = torch.device('cuda:1')

In [8]:
loss_fn = nn.CrossEntropyLoss()

In [9]:
model = torchvision.models.resnet50(weights='DEFAULT')
model.fc = nn.Linear(model.fc.in_features, num_classes, bias=True)
model = model.to(device)

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [11]:
n_epochs = 6
for epoch in tqdm(range(n_epochs), desc='Epoch'):
    model.train()
    losses_trn = []
    for inp, tgt in tqdm(dl_trn, desc='Train', leave=False):
        optimizer.zero_grad()
        out = model(inp.to(device))
        loss = loss_fn(out, tgt.to(device))
        loss.backward()
        optimizer.step()
        losses_trn.append(loss.item())
    
    model.eval()
    losses_test = []
    tgts, preds = [], []
    for inp, tgt in tqdm(dl_test, desc='Test', leave=False):
        with torch.no_grad():
            out = model(inp.to(device))
            loss = loss_fn(out, tgt.to(device))
            tgts.append(tgt)
            preds.append(out.argmax(dim=1).detach().cpu())
        losses_test.append(loss.item())
    tgts = torch.cat(tgts, dim=0)
    preds = torch.cat(preds, dim=0)
    accs = torch.stack([(preds == tgts)[tgts == c].float().mean() for c in range(num_classes)])
    b_acc = accs.mean()
    acc = (preds == tgts).float().mean() 
    
    print(epoch, mean(losses_trn), mean(losses_test), acc, b_acc)

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Train:   0%|          | 0/1562 [00:00<?, ?it/s]

Test:   0%|          | 0/157 [00:00<?, ?it/s]

0 1.0917961484873036 0.8642671035638281 tensor(0.8016) tensor(0.8016)


Train:   0%|          | 0/1562 [00:00<?, ?it/s]

...

Test:   0%|          | 0/157 [00:00<?, ?it/s]

4 0.17838344157872563 0.4281387247536687 tensor(0.8714) tensor(0.8714)


Train:   0%|          | 0/1562 [00:00<?, ?it/s]

Test:   0%|          | 0/157 [00:00<?, ?it/s]

5 0.13233408806192665 0.45685427517268307 tensor(0.8661) tensor(0.8661)


In [12]:
def collect_preds(dl, model):
    model.eval()
    outs, tgts = [], []
    for inp, tgt in dl:
        with torch.no_grad():
            out = model(inp.to(device))
        outs.append(out.detach().cpu())
        tgts.append(tgt)
    tgts = torch.cat(tgts, dim=0)
    outs = torch.cat(outs, dim=0)
    return outs, tgts

In [13]:
# gaussian blur
tfm = TF.Compose([TF.ToTensor(), TF.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
ds_blur  = torchvision.datasets.ImageFolder('./Cifar10/blur/', transform=tfm)
assert len(ds_blur) == 10_000
dl_blur = torch.utils.data.DataLoader(ds_blur, batch_size=64, shuffle=False, drop_last=False)

In [14]:
# gaussian noise
noise_tfm = TF.Compose([tfm, TF2.GaussianNoise(mean=0., sigma=0.01)])
ds_noise = torchvision.datasets.ImageFolder('./Cifar10/test', transform=noise_tfm)
assert len(ds_noise) == 10_000
dl_noise = torch.utils.data.DataLoader(ds_noise, batch_size=64, shuffle=False, drop_last=False)

In [15]:
outs_orig, tgts_orig = collect_preds(dl_test, model)
outs_blur, tgts_blur = collect_preds(dl_blur, model)
outs_noise, tgts_noise = collect_preds(dl_noise, model)

In [16]:
assert (tgts_orig == tgts_blur).all()
assert (tgts_orig == tgts_noise).all()

In [17]:
tgts = tgts_orig

In [18]:
def compute_conf_score(outs):
    probs = outs.softmax(dim=1)
    return probs.max(dim=1).values

def compute_neg_entropy_score(outs):
    probs = outs.softmax(dim=1)
    entropies = probs * probs.log()
    return entropies.sum(dim=1)

In [19]:
def compute_OOD_det_score(outs_ID, outs_OOD, score_fn):
    scores_ID, scores_OOD = score_fn(outs_ID), score_fn(outs_OOD)
    scores_ID, scores_OOD = scores_ID.numpy(), scores_OOD.numpy()
    labels_ID = np.ones((len(scores_ID),), dtype=int)
    labels_OOD = np.zeros((len(scores_OOD),), dtype=int)
    scores = np.concatenate((scores_ID, scores_OOD), axis=0)
    labels = np.concatenate((labels_ID, labels_OOD), axis=0)
    return labels, scores

In [20]:
outs_OOD = {'noise': outs_noise, 'blur': outs_blur}
score_fns = {'conf': compute_conf_score, 'negH': compute_neg_entropy_score}

In [21]:
for corr_name, outs in outs_OOD.items():
    for score_name, score_fn in score_fns.items():
        rocauc = roc_auc_score(*compute_OOD_det_score(outs_orig, outs, score_fn))
        print(corr_name, score_name, f'{rocauc*100:.2f}%')

noise conf 77.15%
noise negH 78.21%
blur conf 72.54%
blur negH 73.09%


In [22]:
torch.save(model, 'ood.pt')